In [14]:
import pandas as pd
import duckdb as db

from plotnine import *
pd.set_option('display.max_columns', None)

In [13]:
df = pd.read_parquet('../data/03_primary/core_applicants_jobs_prospects.parquet', )

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56539 entries, 0 to 56538
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   vaga_id                       56539 non-null  int32 
 1   titulo_vaga                   56539 non-null  object
 2   cliente                       56539 non-null  object
 3   vaga_sap                      56539 non-null  int32 
 4   nivel_profissional_vaga       56539 non-null  object
 5   nivel_academico_vaga          56539 non-null  object
 6   nivel_ingles_vaga             56539 non-null  object
 7   nivel_espanhol_vaga           56539 non-null  object
 8   areas_atuacao                 56539 non-null  object
 9   principais_atividades         56539 non-null  object
 10  competencias                  56539 non-null  object
 11  estado                        56539 non-null  object
 12  cidade                        56539 non-null  object
 13  prospects       

### Verificando se o JOIN está correto. 

> Utilização: Por exemplo, a vaga 10976 (chave no Jobs.json), possui 25 prospecções (chave 10976 no prospects.json), onde o candidato “Sr. Thales Freitas”  (chave 41496 no applicants.json) foi contratado.

In [16]:
df.query("vaga_id == 10976")

,vaga_id,titulo_vaga,cliente,vaga_sap,nivel_profissional_vaga,nivel_academico_vaga,nivel_ingles_vaga,nivel_espanhol_vaga,areas_atuacao,principais_atividades,competencias,estado,cidade,prospects,codigo,nome_candidato,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,prospect_codigo,nome,email,area_atuacao,nivel_profissional_candidato,nivel_academico_candidato,nivel_ingles_candidato,nivel_espanhol_candidato,conhecimentos_tecnicos,cv_pt,target_contratado
41394,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41531,Sra. Beatriz Teixeira,Inscrito,03-02-2023,,,Caroline Machado,41531,Sra. Beatriz Teixeira,sra._beatriz_teixeira@gmail.com,Gestão e Alocação de Recursos de TI,,Pós Graduação Cursando,Intermediário,Nenhum,,"\nbrasileira, solteira, 23 anos.\ncampinas, sã...",0
41395,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41527,Sr. Breno Oliveira,Prospect,03-02-2023,03-02-2023,,Melina Montenegro,41527,Sr. Breno Oliveira,sr._breno_oliveira@hotmail.com,TI - Suporte,,Ensino Superior Completo,Básico,Básico,,experiência\nletiel\nmitsuo analista noc n2/ho...,0
41396,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41507,Dr. Brayan Rezende,Prospect,02-02-2023,02-02-2023,,Maria Clara Pires,41507,Dr. Brayan Rezende,dr._brayan_rezende@gmail.com,,,,,,,resumo\niniciei a minha trajetória profissiona...,0
41397,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41506,Dr. Theo Fernandes,Prospect,02-02-2023,02-02-2023,,Maria Clara Pires,41506,Dr. Theo Fernandes,dr._theo_fernandes@gmail.com,,,,,,,"resumo\nobjetivos profissionais\ninformática, ...",0
41398,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41502,Dr. Yan Vargas,Prospect,02-02-2023,02-02-2023,,Maria Clara Pires,41502,Dr. Yan Vargas,dr._yan_vargas@hotmail.com,,,,,,,\n\nanalista de monitoramento\n\ncódigo apinfo...,0
41399,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41501,Sr. Bruno Rocha,Prospect,02-02-2023,02-02-2023,,Maria Clara Pires,41501,Sr. Bruno Rocha,sr._bruno_rocha@gmail.com,,,,,,,"resumo\nobjetivos profissionais\ninformática, ...",0
41400,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41498,Sr. João Viana,Encaminhado ao Requisitante,02-02-2023,02-02-2023,,Melina Montenegro,41498,Sr. João Viana,sr._joão_viana@hotmail.com,"TI - Qualidade/Testes, TI - Suporte, TI - Telecom",,Ensino Técnico Cursando,Intermediário,Avançado,,nascimento: 13/05/1992\nbairro: fazenda da jut...,0
41401,10976,Analista NOC,Porter-Wilson,0,Analista,Ensino Superior Completo,Básico,Básico,TI - Suporte-,1 analista de N1 NOC para trabalhar de terça a...,1 analista de N1 NOC para trabalhar de terça a...,São Paulo,São Paulo,10976,41496,Sr. Thales Freitas,Contratado pela Decision,02-02-2023,08-02-2023,,Melina Montenegro,41496,Sr. Thales Freitas,sr._thales_freitas@gmail.com,,,,,,,\n\n\ncont

# Verificando os tipos de situação de candidatos


In [8]:
df.situacao_candidado.value_counts()

situacao_candidado
Prospect                          20005
Encaminhado ao Requisitante       16120
Inscrito                           3978
Não Aprovado pelo Cliente          3489
Contratado pela Decision           2757
Desistiu                           2349
Não Aprovado pelo RH               1765
Não Aprovado pelo Requisitante      765
Entrevista Técnica                  579
Sem interesse nesta vaga            576
Entrevista com Cliente              469
Em avaliação pelo RH                375
Contratado como Hunting             226
Aprovado                            209
Desistiu da Contratação              59
Documentação PJ                       4
Documentação CLT                      3
Recusado                              2
Documentação Cooperado                2
Encaminhar Proposta                   2
Proposta Aceita                       1
Name: count, dtype: int64